In [ ]:
Appendix F}
\begin{lstlisting}[language=R]

MIDB <- read.csv("MIDB 5.0.csv")
\\attach(MIDB)
 \\View(MIDB)
 \\MIDB[,c("ccode", "stday", "stmon", "endday", "endmon", "sidea", "revstate", "revtype1", "revtype2", "fatality", "fatalpre", "hiact", "hostlev", "orig", "version")] <- list(NULL)
 \\View(MIDB)
 \\library(tidyverse)
 \\MIDB2<- MIDB %>% 
  \\ mutate(year = map2(styear, endyear, `:`)) %>% 
   \\select(-styear, -endyear) %>% 
   \\unnest
\\# Other option
\\# MIDB2 =  MIDB %>%
\\#  +     rowwise() %>%
\\#  +     mutate(year = list(seq(styear, endyear, 1))) %>%
\\#  +     ungroup() %>%
\\#  +     select(-styear, -endyear) %>%
\\#  +     unnest()
 \\attach(MIDB2)
 \\MIDBTable <- table(stabb,year)
 \\MIDBTable2 <-xtabs(~stabb+year)
 \\write.csv(MIDBTable2, "MIDBTABLE.csv")
 \\MIDBFREQ<-read.csv("MIDBTABLE.csv")
 \\View(MIDBFREQ)
 \\#add missing years (where nothing happens) in excel by hand
 
 \\MIDBTable3<-as.data.frame(MIDBTable)
\\ MIDBTable3$Freq<-ifelse(MIDBTable3$Freq>0,1,0)
\\ MIDBDUMMYTABLE<-xtabs(MIDBTable3$Freq~MIDBTable3$stabb+MIDBTable3$year)
\\ write.csv(MIDBDUMMYTABLE,"DUMMYTABLE.csv")
 \\#again add missing years by hand in excel
\end{lstlisting}

\section*{Appendix G}

\begin{lstlisting}[language=Python]

###### ARIMA MODELS #######################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score as r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.ar_model import ar_select_order
from sklearn.metrics import r2_score
from statsmodels.tsa.arima.model import ARIMA

df_MIDB = pd.read_csv('BINARYTABLE.csv', sep=";")

# df_MIDB

country = 'USA'

## ACF & PACF

plot_acf(df_MIDB[country], lags=20)
plt.show()

plot_pacf(df_MIDB[country], lags=20)
plt.show()


## Stationarity Test ADF ##################################

class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        
    def ADF_Stationarity_Test(self, timeseries, printResults = True):
        #Dickey-Fuller test:
        adfTest = adfuller(timeseries, autolag='AIC')
        
        self.pValue = adfTest[1]
        
        if (self.pValue<self.SignificanceLevel):
            self.isStationary = True
        else:
            self.isStationary = False
        
        if printResults:
            dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adfTest[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)

sTest = StationarityTests(significance=0.05)
sTest.ADF_Stationarity_Test(df_MIDB[country], printResults = True)
print("Is the {} time series stationary? {}".format(country, sTest.isStationary))
print()


## AR ###################################

lags = [4]
res = AutoReg(df_MIDB[country], lags=lags, old_names=False).fit()
print(res.summary())


forecast = res.predict(start= len(df_MIDB), end=len(df_MIDB) + 5)
model_estimate = res.predict(start= 0, end=len(df_MIDB))

plt.plot(model_estimate)
plt.plot(df_MIDB[country])
plt.plot(forecast)
plt.legend(["Model estimate", "True Data", "Forecast"])
plt.show()
print(forecast)


## ARIMA with lags from plots ACF, PACF #####################

country_arima = df_MIDB[country]

model_arima = ARIMA(country_arima, order = (4,0,0)).fit()
print(model_arima.summary())


forecast_arima = model_arima.predict(start= len(df_MIDB), end=len(df_MIDB) + 5)
arima_estimate = model_arima.predict(start= 1, end=len(df_MIDB))


plt.plot(arima_estimate)
plt.plot(country_arima)
plt.plot(forecast_arima)
plt.legend(["Model estimate", "True Data", "Forecast"])
plt.show()
from sklearn.metrics import r2_score
r2 = r2_score(country_arima,arima_estimate)
print('r2: %f' % r2)
forecast_arima


### ARIMA with arbitrary initial lags = 5 #####################

country_arima = df_MIDB[country]

model_arima = ARIMA(country_arima, order = ((1,1,0,1,0),0,(1,1,0,1,0))).fit()
print(model_arima.summary())


forecast_arima = model_arima.predict(start= len(df_MIDB), end=len(df_MIDB) + 5)
arima_estimate = model_arima.predict(start= 1, end=len(df_MIDB))


plt.plot(arima_estimate)
plt.plot(country_arima)
plt.plot(forecast_arima)
plt.legend(["Model estimate", "True Data", "Forecast"])
plt.show()
plt.savefig(country + 'arma_pred.png')

from sklearn.metrics import r2_score
r2 = r2_score(country_arima,arima_estimate)
print('r2: %f' % r2)
forecast_arima


### LOGIT #######################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score as r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import patsy as patsy
from patsy import ModelDesc
from patsy import dmatrices
from patsy import ModelDesc, Term, EvalFactor
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit
from statsmodels.discrete.discrete_model import Probit
import operator
import math
import seaborn as sns
import statsmodels


## Data #########################

df_MIDBFR = pd.read_csv('frequencyMID.csv',sep=";")
df_MIDB = pd.read_csv('BINARYTABLE.csv', sep=";")
Year = df_MIDBFR['YEAR']

## Lagged Variables #############

country = 'USA'
FirstPredictedYear = 2015
row= FirstPredictedYear - 1816
row2= row + 1
row3= row2 + 1
row4= row3 + 1
row5= row4 + 1
inter_confl = df_MIDB[country]
s3 = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan])
inter_confl=inter_confl.append(s3,ignore_index=True)
inter_confl=inter_confl.rename(country + "Conflict")

inter_freq = df_MIDBFR[country]
s3 = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan])
inter_freq=inter_freq.append(s3,ignore_index=True)
inter_freq=inter_freq.rename(country + "Frequency")
inter_freq1 = inter_freq.shift(1)
inter_freq1 = inter_freq1.rename("L1_"+ country +"Frequency")
inter_freq2 = inter_freq1.shift(1)
inter_freq2 = inter_freq2.rename("L2_"+ country +"Frequency")
inter_freq3 = inter_freq2.shift(1)
inter_freq3 = inter_freq3.rename("L3_"+ country +"Frequency")
inter_freq4 = inter_freq3.shift(1)
inter_freq4 = inter_freq4.rename("L4_"+ country +"Frequency")
inter_freq5 = inter_freq4.shift(1)
inter_freq5 = inter_freq5.rename("L5_"+ country +"Frequency")

inter_lagged = inter_confl.shift(1)
inter_lagged = inter_lagged.rename("L1_" + country + "Conflict")
inter_lagged2 = inter_lagged.shift(1)
inter_lagged2 = inter_lagged2.rename("L2_"+ country +"Conflict")
inter_lagged3 = inter_lagged2.shift(1)
inter_lagged3 = inter_lagged3.rename("L3_"+ country +"Conflict")
inter_lagged4 = inter_lagged3.shift(1)
inter_lagged4 = inter_lagged4.rename("L4_"+ country +"Conflict")
inter_lagged5 = inter_lagged4.shift(1)
inter_lagged5 = inter_lagged5.rename("L5_"+ country +"Conflict")
data_confl = pd.concat([inter_confl, inter_lagged, inter_lagged2, inter_lagged3, inter_lagged4, inter_lagged5, inter_freq, inter_freq1, inter_freq2, inter_freq3, inter_freq4, inter_freq5],axis=1)


## Poisson Regression for Frequency ####################

## Poisson to predict frequency of unobserved year #####

Z, K = dmatrices(country + 'Frequency'+ '~' + 'L1_'+country+'Frequency + L2_' +country+'Frequency + L3_' +country+ 'Frequency + L4_' 
                 +country+ 'Frequency + L5_' +country+ 'Frequency', NA_action=patsy.NAAction(NA_types=[]), data=data_confl, return_type='dataframe')

def remove_most_insignificant(df, results):
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    df.drop(columns = max_p_value, inplace = True)
    return df

insignificant_feature = True
while insignificant_feature:
    modelFR = sm.Poisson(Z, K,missing='drop')
    resultsFR = modelFR.fit(cov_type='HC3')
    significant = [p_value < 0.05 for p_value in resultsFR.pvalues[1:]]
    if all(significant):
        insignificant_feature = False
    else:
        if K.shape[1] == 1:
            print('No significant features found')
            resultsFR = None
            insignificant_feature = False
        else:
            K = remove_most_insignificant(K, resultsFR)

print(resultsFR.summary())

signif_valuesFR = resultsFR.params.to_frame()
signif_valuesFR = signif_valuesFR.reset_index()
signif_valuesFR.columns = ['sign_variable','coef']


zeta_1 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L1_'+ country + 'Frequency'] ['coef'].values
zeta_2 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L2_'+ country + 'Frequency'] ['coef'].values
zeta_3 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L3_'+ country + 'Frequency'] ['coef'].values
zeta_4 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L4_'+ country + 'Frequency'] ['coef'].values
zeta_5 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L5_'+ country + 'Frequency'] ['coef'].values

interceptFR = signif_valuesFR[signif_valuesFR['sign_variable'] == 'Intercept'] ['coef'].values
if zeta_1.size <= 0:
    zeta_1 = 0

if zeta_2.size <= 0:
    zeta_2 = 0

if zeta_3.size <= 0:
    zeta_3 = 0

if zeta_4.size <= 0:
    zeta_4 = 0

if zeta_5.size <= 0:
    zeta_5 = 0

Frequency1year = interceptFR + (zeta_1 * data_confl['L1_' +country+ 'Frequency'][row]) + (zeta_2 * data_confl['L2_' +country+ 'Frequency'][row]) + (zeta_3 * data_confl['L3_' +country+ 'Frequency'][row]) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row])
Frequency1year = math.exp(Frequency1year)
Frequency2year = interceptFR + (zeta_1 * Frequency1year) + (zeta_2 * data_confl['L2_' +country+ 'Frequency'][row2]) + (zeta_3 * data_confl['L3_' +country+ 'Frequency'][row2]) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row2]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row2])
Frequency2year = math.exp(Frequency2year)
Frequency3year = interceptFR + (zeta_1 * Frequency2year) + (zeta_2 * Frequency1year) + (zeta_3 * data_confl['L3_' +country+ 'Frequency'][row3]) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row3]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row3])
Frequency3year = math.exp(Frequency3year)
Frequency4year = interceptFR + (zeta_1 * Frequency3year) + (zeta_2 * Frequency2year) + (zeta_3 * Frequency1year) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row4]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row4])
Frequency4year = math.exp(Frequency4year)
Frequency5year = interceptFR + (zeta_1 * Frequency4year) + (zeta_2 * Frequency3year) + (zeta_3 * Frequency2year) + (zeta_4 * Frequency1year) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row5])
Frequency5year = math.exp(Frequency5year)
print(Frequency1year)
print(Frequency2year)
print(Frequency3year)
print(Frequency4year)
print(Frequency5year)


## Logit ######################

#Now that we have all data we can use LOGIT to predict
Y, X = dmatrices(country + 'Conflict'+ '~' + ' L1_'+ country + 'Conflict + L2_'+ country + 'Conflict + L3_'+ country +'Conflict + L4_'+ country +
                 'Conflict + L5_'+ country +'Conflict + L1_'+country+'Frequency + L2_' +country+'Frequency + L3_' +country+ 'Frequency + L4_' 
                 +country+ 'Frequency + L5_' +country+ 'Frequency', NA_action=patsy.NAAction(NA_types=[]), data=data_confl, return_type='dataframe')



def remove_most_insignificant(df, results):
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    df.drop(columns = max_p_value, inplace = True)
    return df

insignificant_feature = True
while insignificant_feature:
    model = sm.Logit(Y, X,missing='drop')
    results = model.fit()
    significant = [p_value < 0.05 for p_value in results.pvalues[1:]]
    if all(significant):
        insignificant_feature = False
    else:
        if X.shape[1] == 1:
            print('No significant features found')
            results = None
            insignificant_feature = False
        else:
            X = remove_most_insignificant(X, results)

print(results.summary())

signif_values = results.params.to_frame()
signif_values = signif_values.reset_index()
signif_values.columns = ['sign_variable','coef']
beta1 = signif_values[signif_values['sign_variable'] == 'L1_' + country + 'Conflict'] ['coef'].values
beta2 = signif_values[signif_values['sign_variable'] == 'L2_' + country + 'Conflict'] ['coef'].values
beta3 = signif_values[signif_values['sign_variable'] == 'L3_' + country + 'Conflict'] ['coef'].values
beta4 = signif_values[signif_values['sign_variable'] == 'L4_' + country + 'Conflict'] ['coef'].values
beta5 = signif_values[signif_values['sign_variable'] == 'L5_' + country + 'Conflict'] ['coef'].values

theta1 = signif_values[signif_values['sign_variable'] == 'L1_'+ country + 'Frequency'] ['coef'].values
theta2 = signif_values[signif_values['sign_variable'] == 'L2_'+ country + 'Frequency'] ['coef'].values
theta3 = signif_values[signif_values['sign_variable'] == 'L3_'+ country + 'Frequency'] ['coef'].values
theta4 = signif_values[signif_values['sign_variable'] == 'L4_'+ country + 'Frequency'] ['coef'].values
theta5 = signif_values[signif_values['sign_variable'] == 'L5_'+ country + 'Frequency'] ['coef'].values

intercept = signif_values[signif_values['sign_variable'] == 'Intercept'] ['coef'].values

if theta1.size <= 0:
    theta1 = 0
    
if theta2.size <= 0:
    theta2 = 0
    
if theta3.size <= 0:
    theta3 = 0
    
if theta4.size <= 0:
    theta4 = 0
    
if theta5.size <= 0:
    theta5 = 0

if beta1.size <= 0:
    beta1 = 0
    
if beta2.size <= 0:
    beta2 = 0
    
if beta3.size <= 0:
    beta3 = 0
    
if beta4.size <= 0:
    beta4 = 0
    
if beta5.size <= 0:
    beta5 = 0

Y1year = intercept + beta1 * data_confl['L1_' +country+ 'Conflict'][row] + beta2 * data_confl['L2_' +country+ 'Conflict'][row] + beta3 * data_confl['L3_' +country+ 'Conflict'][row] + beta4 * data_confl['L4_' +country+ 'Conflict'][row] + beta5 * data_confl['L5_' +country+ 'Conflict'][row] + theta1 * data_confl['L1_' +country+ 'Frequency'][row] + theta2 * data_confl['L2_' +country+ 'Frequency'][row] + theta3 * data_confl['L3_' +country+ 'Frequency'][row] + theta4 * data_confl['L4_' +country+ 'Frequency'][row] + theta5 * data_confl['L5_' +country+ 'Frequency'][row]
P1year = math.exp(Y1year)/(1+(math.exp(Y1year)))
Y2year = intercept + beta1 * P1year + beta2 * data_confl['L2_' +country+ 'Conflict'][row2] + beta3 * data_confl['L3_' +country+ 'Conflict'][row2] + beta4 * data_confl['L4_' +country+ 'Conflict'][row2] + beta5 * data_confl['L5_' +country+ 'Conflict'][row2] + theta1 * Frequency1year + theta2 * data_confl['L2_' +country+ 'Frequency'][row2] + theta3 * data_confl['L3_' +country+ 'Frequency'][row2] + theta4 * data_confl['L4_' +country+ 'Frequency'][row2] + theta5 * data_confl['L5_' +country+ 'Frequency'][row2]
P2year = math.exp(Y2year)/(1+(math.exp(Y2year)))
Y3year = intercept + beta1 * P2year + beta2 * P1year + beta3 * data_confl['L3_' +country+ 'Conflict'][row3] + beta4 * data_confl['L4_' +country+ 'Conflict'][row3] + beta5 * data_confl['L5_' +country+ 'Conflict'][row3] + theta1 * Frequency2year + theta2 * Frequency1year + theta3 * data_confl['L3_' +country+ 'Frequency'][row3] + theta4 * data_confl['L4_' +country+ 'Frequency'][row3] + theta5 * data_confl['L5_' +country+ 'Frequency'][row3]
P3year = math.exp(Y3year)/(1+(math.exp(Y3year)))
Y4year = intercept + beta1 * P3year + beta2 * P2year + beta3 * P1year + beta4 * data_confl['L4_' +country+ 'Conflict'][row4] + beta5 * data_confl['L5_' +country+ 'Conflict'][row4] + theta1 * Frequency3year + theta2 * Frequency2year + theta3 * Frequency1year + theta4 * data_confl['L4_' +country+ 'Frequency'][row4] + theta5 * data_confl['L5_' +country+ 'Frequency'][row4]
P4year = math.exp(Y4year)/(1+(math.exp(Y4year)))
Y5year = intercept + beta1 * P4year + beta2 * P3year + beta3 * P2year + beta4 * P1year + beta5 * data_confl['L5_' +country+ 'Conflict'][row5] + theta1 * Frequency4year + theta2 * Frequency3year + theta3 * Frequency2year + theta4 * Frequency1year + theta5 * data_confl['L5_' +country+ 'Frequency'][row5]
P5year = math.exp(Y5year)/(1+(math.exp(Y5year)))

# probabilities #######

print(P1year)
print(P2year)
print(P3year)
print(P4year)
print(P5year)

## AIC & BIC #####

print(results.aic)
print(results.bic)


pred=results.predict()
preds=pd.DataFrame(pred)
forecast1=[P1year,P2year,P3year,P4year,P5year]
preds = preds.append(forecast1,ignore_index=True)
startValue=204-len(preds)


realData=df_MIDB[country]
indexYear = pd.read_csv('YearIndex.csv')[startValue:]
index2 = df_MIDB['YEAR'][:]
wide_df2 = pd.DataFrame(realData)
plt.figure(figsize=(20,8))
sns.scatterplot(y=realData,x=index2)
sns.regplot(x=indexYear,y=preds,logistic=True,scatter=True,color='red')
plt.title('Regression Line of Conflict for ' + country + ' Logit')
plt.xlabel('Year') 
plt.ylabel('probabilities')
plt.savefig(country + '_logit_predict.png')


preds2 = [P1year,P2year,P3year,P4year,P5year]
x = ['2015','2016','2017','2018','2019']
plt.title('Forecasted probabilities for 5 years ' + country + ' Logit')
plt.xlabel('Year') 
plt.ylabel('Probabilities')
plt.grid()
plt.plot(x,preds2, 'o-', markeredgewidth=0)
plt.savefig(country + '_5years_predict_logit.png')


## PROBIT #################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score as r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import patsy as patsy
from patsy import ModelDesc
from patsy import dmatrices
from patsy import ModelDesc, Term, EvalFactor
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit
from statsmodels.discrete.discrete_model import Probit
import operator
import math
import seaborn as sns
import statsmodels

## Data ###############

df_MIDBFR = pd.read_csv('frequencyMID.csv',sep=";")
df_MIDB = pd.read_csv('BINARYTABLE.csv', sep=";")
Year = df_MIDBFR['YEAR']

## Lagged Variables ###########

country = 'USA'
FirstPredictedYear = 2015
row= FirstPredictedYear - 1816
row2= row + 1
row3= row2 + 1
row4= row3 + 1
row5= row4 + 1
inter_confl = df_MIDB[country]
s3 = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan])
inter_confl=inter_confl.append(s3,ignore_index=True)
inter_confl=inter_confl.rename(country + "Conflict")

inter_freq = df_MIDBFR[country]
s3 = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan])
inter_freq=inter_freq.append(s3,ignore_index=True)
inter_freq=inter_freq.rename(country + "Frequency")
inter_freq1 = inter_freq.shift(1)
inter_freq1 = inter_freq1.rename("L1_"+ country +"Frequency")
inter_freq2 = inter_freq1.shift(1)
inter_freq2 = inter_freq2.rename("L2_"+ country +"Frequency")
inter_freq3 = inter_freq2.shift(1)
inter_freq3 = inter_freq3.rename("L3_"+ country +"Frequency")
inter_freq4 = inter_freq3.shift(1)
inter_freq4 = inter_freq4.rename("L4_"+ country +"Frequency")
inter_freq5 = inter_freq4.shift(1)
inter_freq5 = inter_freq5.rename("L5_"+ country +"Frequency")

inter_lagged = inter_confl.shift(1)
inter_lagged = inter_lagged.rename("L1_" + country + "Conflict")
inter_lagged2 = inter_lagged.shift(1)
inter_lagged2 = inter_lagged2.rename("L2_"+ country +"Conflict")
inter_lagged3 = inter_lagged2.shift(1)
inter_lagged3 = inter_lagged3.rename("L3_"+ country +"Conflict")
inter_lagged4 = inter_lagged3.shift(1)
inter_lagged4 = inter_lagged4.rename("L4_"+ country +"Conflict")
inter_lagged5 = inter_lagged4.shift(1)
inter_lagged5 = inter_lagged5.rename("L5_"+ country +"Conflict")
data_confl = pd.concat([inter_confl, inter_lagged, inter_lagged2, inter_lagged3, inter_lagged4, inter_lagged5, inter_freq, inter_freq1, inter_freq2, inter_freq3, inter_freq4, inter_freq5],axis=1)


### Poisson Regression for Frequency #################

# Poisson to predict frequency of unobserved year ######

Z, K = dmatrices(country + 'Frequency'+ '~' + 'L1_'+country+'Frequency + L2_' +country+'Frequency + L3_' +country+ 'Frequency + L4_' 
                 +country+ 'Frequency + L5_' +country+ 'Frequency', NA_action=patsy.NAAction(NA_types=[]), data=data_confl, return_type='dataframe')

def remove_most_insignificant(df, results):
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    df.drop(columns = max_p_value, inplace = True)
    return df

insignificant_feature = True
while insignificant_feature:
    modelFR = sm.Poisson(Z, K,missing='drop')
    resultsFR = modelFR.fit(cov_type='HC3')
    significant = [p_value < 0.05 for p_value in resultsFR.pvalues[1:]]
    if all(significant):
        insignificant_feature = False
    else:
        if K.shape[1] == 1:
            print('No significant features found')
            resultsFR = None
            insignificant_feature = False
        else:
            K = remove_most_insignificant(K, resultsFR)

print(resultsFR.summary())

signif_valuesFR = resultsFR.params.to_frame()
signif_valuesFR = signif_valuesFR.reset_index()
signif_valuesFR.columns = ['sign_variable','coef']


zeta_1 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L1_'+ country + 'Frequency'] ['coef'].values
zeta_2 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L2_'+ country + 'Frequency'] ['coef'].values
zeta_3 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L3_'+ country + 'Frequency'] ['coef'].values
zeta_4 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L4_'+ country + 'Frequency'] ['coef'].values
zeta_5 = signif_valuesFR[signif_valuesFR['sign_variable'] == 'L5_'+ country + 'Frequency'] ['coef'].values

interceptFR = signif_valuesFR[signif_valuesFR['sign_variable'] == 'Intercept'] ['coef'].values
if zeta_1.size <= 0:
    zeta_1 = 0

if zeta_2.size <= 0:
    zeta_2 = 0

if zeta_3.size <= 0:
    zeta_3 = 0

if zeta_4.size <= 0:
    zeta_4 = 0

if zeta_5.size <= 0:
    zeta_5 = 0

Frequency1year = interceptFR + (zeta_1 * data_confl['L1_' +country+ 'Frequency'][row]) + (zeta_2 * data_confl['L2_' +country+ 'Frequency'][row]) + (zeta_3 * data_confl['L3_' +country+ 'Frequency'][row]) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row])
Frequency1year = math.exp(Frequency1year)
Frequency2year = interceptFR + (zeta_1 * Frequency1year) + (zeta_2 * data_confl['L2_' +country+ 'Frequency'][row2]) + (zeta_3 * data_confl['L3_' +country+ 'Frequency'][row2]) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row2]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row2])
Frequency2year = math.exp(Frequency2year)
Frequency3year = interceptFR + (zeta_1 * Frequency2year) + (zeta_2 * Frequency1year) + (zeta_3 * data_confl['L3_' +country+ 'Frequency'][row3]) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row3]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row3])
Frequency3year = math.exp(Frequency3year)
Frequency4year = interceptFR + (zeta_1 * Frequency3year) + (zeta_2 * Frequency2year) + (zeta_3 * Frequency1year) + (zeta_4 * data_confl['L4_' +country+ 'Frequency'][row4]) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row4])
Frequency4year = math.exp(Frequency4year)
Frequency5year = interceptFR + (zeta_1 * Frequency4year) + (zeta_2 * Frequency3year) + (zeta_3 * Frequency2year) + (zeta_4 * Frequency1year) + (zeta_5 * data_confl['L5_' +country+ 'Frequency'][row5])
Frequency5year = math.exp(Frequency5year)
print(Frequency1year)
print(Frequency2year)
print(Frequency3year)
print(Frequency4year)
print(Frequency5year)

## Probit ##########################

#Now that we have all data we can use PROBIT to predict##
Y, X = dmatrices(country + 'Conflict'+ '~' + ' L1_'+ country + 'Conflict + L2_'+ country + 'Conflict + L3_'+ country +'Conflict + L4_'+ country +
                 'Conflict + L5_'+ country +'Conflict + L1_'+country+'Frequency + L2_' +country+'Frequency + L3_' +country+ 'Frequency + L4_' 
                 +country+ 'Frequency + L5_' +country+ 'Frequency', NA_action=patsy.NAAction(NA_types=[]), data=data_confl, return_type='dataframe')


def remove_most_insignificant(df, results):
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    df.drop(columns = max_p_value, inplace = True)
    return df

insignificant_feature = True
while insignificant_feature:
    model = sm.Probit(Y, X,missing='drop')
    results = model.fit()
    significant = [p_value < 0.05 for p_value in results.pvalues[1:]]
    if all(significant):
        insignificant_feature = False
    else:
        if X.shape[1] == 1:
            print('No significant features found')
            results = None
            insignificant_feature = False
        else:
            X = remove_most_insignificant(X, results)

print(results.summary())

signif_values = results.params.to_frame()
signif_values = signif_values.reset_index()
signif_values.columns = ['sign_variable','coef']
beta1 = signif_values[signif_values['sign_variable'] == 'L1_' + country + 'Conflict'] ['coef'].values
beta2 = signif_values[signif_values['sign_variable'] == 'L2_' + country + 'Conflict'] ['coef'].values
beta3 = signif_values[signif_values['sign_variable'] == 'L3_' + country + 'Conflict'] ['coef'].values
beta4 = signif_values[signif_values['sign_variable'] == 'L4_' + country + 'Conflict'] ['coef'].values
beta5 = signif_values[signif_values['sign_variable'] == 'L5_' + country + 'Conflict'] ['coef'].values

theta1 = signif_values[signif_values['sign_variable'] == 'L1_'+ country + 'Frequency'] ['coef'].values
theta2 = signif_values[signif_values['sign_variable'] == 'L2_'+ country + 'Frequency'] ['coef'].values
theta3 = signif_values[signif_values['sign_variable'] == 'L3_'+ country + 'Frequency'] ['coef'].values
theta4 = signif_values[signif_values['sign_variable'] == 'L4_'+ country + 'Frequency'] ['coef'].values
theta5 = signif_values[signif_values['sign_variable'] == 'L5_'+ country + 'Frequency'] ['coef'].values

intercept = signif_values[signif_values['sign_variable'] == 'Intercept'] ['coef'].values

if theta1.size <= 0:
    theta1 = 0
    
if theta2.size <= 0:
    theta2 = 0
    
if theta3.size <= 0:
    theta3 = 0
    
if theta4.size <= 0:
    theta4 = 0
    
if theta5.size <= 0:
    theta5 = 0

if beta1.size <= 0:
    beta1 = 0
    
if beta2.size <= 0:
    beta2 = 0
    
if beta3.size <= 0:
    beta3 = 0
    
if beta4.size <= 0:
    beta4 = 0
    
if beta5.size <= 0:
    beta5 = 0

import scipy
import scipy.stats as st
Y1year= intercept + beta1 * data_confl['L1_' +country+ 'Conflict'][row] + beta2 * data_confl['L2_' +country+ 'Conflict'][row] + beta3 * data_confl['L3_' +country+ 'Conflict'][row] + beta4 * data_confl['L4_' +country+ 'Conflict'][row] + beta5 * data_confl['L5_' +country+ 'Conflict'][row] + theta1 * data_confl['L1_' +country+ 'Frequency'][row] + theta2 * data_confl['L2_' +country+ 'Frequency'][row] + theta3 * data_confl['L3_' +country+ 'Frequency'][row] + theta4 * data_confl['L4_' +country+ 'Frequency'][row] + theta5 * data_confl['L5_' +country+ 'Frequency'][row]
P1year = st.norm.cdf(Y1year)
Y2year = intercept + beta1 * P1year + beta2 * data_confl['L2_' +country+ 'Conflict'][row2] + beta3 * data_confl['L3_' +country+ 'Conflict'][row2] + beta4 * data_confl['L4_' +country+ 'Conflict'][row2] + beta5 * data_confl['L5_' +country+ 'Conflict'][row2] + theta1 * Frequency1year + theta2 * data_confl['L2_' +country+ 'Frequency'][row2] + theta3 * data_confl['L3_' +country+ 'Frequency'][row2] + theta4 * data_confl['L4_' +country+ 'Frequency'][row2] + theta5 * data_confl['L5_' +country+ 'Frequency'][row2]
P2year = st.norm.cdf(Y2year)
Y3year = intercept + beta1 * P2year + beta2 * P1year + beta3 * data_confl['L3_' +country+ 'Conflict'][row3] + beta4 * data_confl['L4_' +country+ 'Conflict'][row3] + beta5 * data_confl['L5_' +country+ 'Conflict'][row3] + theta1 * Frequency2year + theta2 * Frequency1year + theta3 * data_confl['L3_' +country+ 'Frequency'][row3] + theta4 * data_confl['L4_' +country+ 'Frequency'][row3] + theta5 * data_confl['L5_' +country+ 'Frequency'][row3]
P3year = st.norm.cdf(Y3year)
Y4year = intercept + beta1 * P3year + beta2 * P2year + beta3 * P1year + beta4 * data_confl['L4_' +country+ 'Conflict'][row4] + beta5 * data_confl['L5_' +country+ 'Conflict'][row4] + theta1 * Frequency3year + theta2 * Frequency2year + theta3 * Frequency1year + theta4 * data_confl['L4_' +country+ 'Frequency'][row4] + theta5 * data_confl['L5_' +country+ 'Frequency'][row4]
P4year = st.norm.cdf(Y4year)
Y5year = intercept + beta1 * P4year + beta2 * P3year + beta3 * P2year + beta4 * P1year + beta5 * data_confl['L5_' +country+ 'Conflict'][row5] + theta1 * Frequency4year + theta2 * Frequency3year + theta3 * Frequency2year + theta4 * Frequency1year + theta5 * data_confl['L5_' +country+ 'Frequency'][row5]
P5year = st.norm.cdf(Y5year)
print(P1year)
print(P2year)
print(P3year)
print(P4year)
print(P5year)

## AIC & BIC #############

print(results.aic)
print(results.bic)

pred=results.predict()
preds=pd.DataFrame(pred)
forecast1=[P1year,P2year,P3year,P4year,P5year]
preds = preds.append(forecast1,ignore_index=True)
startValue=204-len(preds)

realData=df_MIDB[country]
indexYear = pd.read_csv('YearIndex.csv')[startValue:]
index2 = df_MIDB['YEAR'][:]
wide_df2 = pd.DataFrame(realData)
plt.figure(figsize=(18,8))
sns.scatterplot(y=realData,x=index2)
sns.regplot(x=indexYear,y=preds,logistic=True,scatter=True,color='red')
plt.title('Regression Line of Conflict for ' + country + ' Probit')
plt.xlabel('Year') 
plt.ylabel('Probabilities')
plt.savefig(country + '_probit_predict.png')

preds2 = [P1year,P2year,P3year,P4year,P5year]
x = ['2015','2016','2017','2018','2019']
plt.title('Forecasted probabilities for 5 years ' + country + ' Probit')
plt.xlabel('Year') 
plt.ylabel('Probabilities')
plt.grid()
plt.plot(x,preds2, 'o-', markeredgewidth=0)
plt.savefig(country + '_5years_predict_probit.png')


from sklearn import metrics

diff = len(data_confl)-5 -len(pred)

metrics1 = metrics.accuracy_score(realData[diff:],pred.round(),normalize=True)
metrics2 = metrics.accuracy_score(realData[diff:],pred.round(),normalize=False)

print(metrics1)
print(metrics2)


\end{lstlisting}

\end{appendices}

%TC:endignore
\end{document}

